In [8]:
#import dependencies 
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time 
import pandas as pd
import re
import time
import numpy as np

In [92]:
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

def scrape():

    browser = init_browser()
    
    #get latest tweet 
    handle = "marswxreport"
    query="InSight"
    
    twitter_url = f'https://twitter.com/{handle}?lang=en'
    
    browser.visit(twitter_url)
    
    time.sleep(3)

    pattern = re.compile(query)

    html_twitter = browser.html
    soup_twitter = bs(html_twitter, 'html.parser')

    tweet = soup_twitter.find('span', text=pattern).text 
    
    #get news title and article
    news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    
    browser.visit(news_url)
    
    time.sleep(3)

    html_news = browser.html
    soup_news = bs(html_news, 'html.parser')

    news_title = soup_news.find('div', class_="list_text").a.text
    news_article = soup_news.find('div', class_="article_teaser_body").text 
    
    #get featured image
    image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    
    browser.visit(image_url)
    
    time.sleep(3)
    
    browser.click_link_by_partial_text('FULL IMAGE')
    
    time.sleep(3)
    
    browser.click_link_by_partial_text('more info')
    
    time.sleep(3)

    html_image = browser.html
    soup_image = bs(html_image, 'html.parser')

    featured_image = soup_image.find('figure', class_='lede').a["href"]
    
    #get facts table
    facts_url = "https://space-facts.com/mars/"

    facts_table = pd.read_html(facts_url)    

    facts_table = facts_table[0]
    facts_table.columns = ["Property", "Value"]
    facts_dict = facts_table.to_dict('index')
    
    
    #store data in a dictionary
    results = {
        'news_title': news_title,
        'news_article': news_article,
        'tweet': tweet,
        'featured_image': featured_image
#         'tweet': tweets,
#         'likes': likes,
#         'comments': comments,
#         'retweets': retweets
    }
    
    results = {**results, **facts_dict}
    
    #close the browser
    browser.quit()
    
    return results


In [93]:
results = scrape()

results

{'news_title': "NASA's Perseverance Rover Mission Getting in Shape for Launch",
 'news_article': 'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. ',
 'tweet': 'InSight sol 514 (2020-05-07) low -92.3ºC (-134.1ºF) high -2.8ºC (27.0ºF)\nwinds from the SW at 5.2 m/s (11.6 mph) gusting to 16.8 m/s (37.5 mph)\npressure at 6.90 hPa',
 'featured_image': '/spaceimages/images/largesize/PIA20318_hires.jpg',
 0: {'Property': 'Equatorial Diameter:', 'Value': '6,792 km'},
 1: {'Property': 'Polar Diameter:', 'Value': '6,752 km'},
 2: {'Property': 'Mass:', 'Value': '6.39 × 10^23 kg (0.11 Earths)'},
 3: {'Property': 'Moons:', 'Value': '2 (Phobos & Deimos)'},
 4: {'Property': 'Orbit Distance:', 'Value': '227,943,824 km (1.38 AU)'},
 5: {'Property': 'Orbit Period:', 'Value': '687 days (1.9 years)'},
 6: {'Property': 'Surface Temperature:', 'Value': '-87 to -5 °C'},
 7: {'Property': 'First Record:', 'Value': '2nd millenniu